In [1]:
import os
import sys
base_module_path = os.path.abspath(os.path.join('..'))
if base_module_path not in sys.path:
    sys.path.append(base_module_path)
import cervix as c

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np 
import keras

In [3]:
train = c.data.make_base_df()
train = c.processing.transform_save_imgs(train, c.processing.resize_n, n=299); train.head()


,path,Type,filetype,num_id,processed_path
0,../data/train/Type_1/0.jpg,1,jpg,0,../data/processed/resize_299/train/0.jpg
1,../data/train/Type_1/10.jpg,1,jpg,10,../data/processed/resize_299/train/10.jpg
2,../data/train/Type_1/1013.jpg,1,jpg,1013,../data/processed/resize_299/train/1013.jpg
3,../data/train/Type_1/1014.jpg,1,jpg,1014,../data/processed/resize_299/train/1014.jpg
4,../data/train/Type_1/1019.jpg,1,jpg,1019,../data/processed/resize_299/train/1019.jpg


In [4]:
test = c.data.make_test_df()
test = c.processing.transform_save_imgs(test, c.processing.resize_n, n=299)

0 1 10 100 101 102 103 104 105 106 107 108 109 11 110 111 112 113 114 115 116 117 118 119 12 120 121 122 123 124 125 126 127 128 129 13 130 131 132 133 134 135 136 137 138 139 14 140 141 142 143 144 145 146 147 148 149 15 150 151 152 153 154 155 156 157 158 159 16 160 161 162 163 164 165 166 167 168 169 17 170 171 172 173 174 175 176 177 178 179 18 180 181 182 183 184 185 186 187 188 189 19 190 191 192 193 194 195 196 197 198 199 2 20 200 201 202 203 204 205 206 207 208 209 21 210 211 212 213 214 215 216 217 218 219 22 220 221 222 223 224 225 226 227 228 229 23 230 231 232 233 234 235 236 237 238 239 24 240 241 242 243 244 245 246 247 248 249 25 250 251 252 253 254 255 256 257 258 259 26 260 261 262 263 264 265 266 267 268 269 27 270 271 272 273 274 275 276 277 278 279 28 280 281 282 283 284 285 286 287 288 289 29 290 291 292 293 294 295 296 297 298 299 3 30 300 301 302 303 304 305 306 307 308 309 31 310 311 312 313 314 315 316 317 318 319 32 320 321 322 323 324 325 326 327 328 329 33 

In [6]:
model_fp = '../data/model/v3_172_SGD_v1.h5'
model = keras.models.load_model(model_fp)

In [10]:
import cv2
def df_to_keras_generator(df, batch_size, grayscale=None):
    assert isinstance(grayscale, (bool)), 'grayscale must be set to a bool'
    imread_opt = 0 if grayscale else 1 # 1 is 3chan rbg, 0 is grayscale

    batch_features = np.zeros((batch_size, 299, 299, 3))

    batch_counter = 0
    while 1:
        for index, row in df.iterrows():
            batch_features[batch_counter] = cv2.imread(row['processed_path'],imread_opt)
            batch_counter += 1
            if batch_counter == batch_size:
                batch_counter = 0
                yield batch_features


In [11]:
batch_size_t = 1
predictions = model.predict_generator(df_to_keras_generator(test, batch_size_t, grayscale=False)
                    ,len(test))

In [12]:
predictions

array([[ 0.17907885,  0.50894713,  0.31197408],
       [ 0.17123672,  0.46972373,  0.35903952],
       [ 0.17067277,  0.45749   ,  0.37183732],
       ..., 
       [ 0.18636882,  0.47871351,  0.3349176 ],
       [ 0.07585896,  0.84225744,  0.08188362],
       [ 0.27098393,  0.44333118,  0.28568494]], dtype=float32)

In [13]:
test = c.processing.append_probabilities(test, predictions, ['1','2','3']); 

In [14]:
test.head()

,path,num_id,processed_path,Type_1,Type_2,Type_3
0,../data/test/0.jpg,0,../data/processed/resize_299/test/0.jpg,0.179079,0.508947,0.311974
1,../data/test/1.jpg,1,../data/processed/resize_299/test/1.jpg,0.171237,0.469724,0.359040
2,../data/test/10.jpg,10,../data/processed/resize_299/test/10.jpg,0.170673,0.457490,0.371837
3,../data/test/100.jpg,100,../data/processed/resize_299/test/100.jpg,0.147985,0.540032,0.311983
4,../data/test/101.jpg,101,../data/processed/resize_299/test/101.jpg,0.229509,0.385721,0.384769


In [15]:
c.submission.write_submission_file('output_v3_172_SGD_v1.csv', test)